# Testing functionality of TKM and JAX

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!export XLA_PYTHON_CLIENT_MEM_FRACTION=.1

In [3]:
import jax
cpu = jax.devices("cpu")[0]
jax.default_device(cpu)

In [2]:
# from jax.config import config
# config.update("jax_enable_x64", True)

## Load Banana Data

In [4]:
import pandas as pd
from pathlib import Path

In [5]:
data_path = Path("../data/banana.csv")
df_banana = pd.read_csv(data_path)

In [6]:
df_banana.head()

,V1,V2,Class
0,1.140,-0.114,1
1,-1.520,-1.150,2
2,-1.050,0.720,1
3,-0.916,0.397,2
4,-1.090,0.437,2


In [7]:
# expecting labels 1 and -1

# transform label 2 to -1
df_banana.loc[df_banana.Class == 2, 'Class'] = -1
df_banana.head()

,V1,V2,Class
0,1.140,-0.114,1
1,-1.520,-1.150,-1
2,-1.050,0.720,1
3,-0.916,0.397,-1
4,-1.090,0.437,-1


In [8]:
# standardize

# df_banana[['V1','V2']] = (df_banana[['V1','V2']] -  df_banana[['V1','V2']].min() )/ (df_banana[['V1','V2']].max() - df_banana[['V1','V2']].min())
# df_banana.head()

In [9]:
feature_names = ['V1', 'V2']
x = df_banana[feature_names].to_numpy()

label_name = ['Class']
y = df_banana[label_name].to_numpy()

In [10]:
# rescale

x = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))

### Data to JAX array

In [11]:
from jax import numpy as jnp
x = jnp.array(x)
y = jnp.array(y)

In [12]:
from jax import random
key = random.PRNGKey(42)

## Tensor Kernel Machine

### Polynomial

In [13]:
from tkm.model import fit
from jax import jit
from tkm.features import polynomial


In [14]:
W = fit(key,x,y,M=12,R=6,feature_map=polynomial)

In [15]:
%timeit fit(key,x,y,M=12,R=6,feature_map=polynomial)

194 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
from functools import partial

fit_polynomial = partial(fit, feature_map=polynomial, M=12, R=6)
fit_compiled = jit(fit_polynomial)

%timeit fit_compiled(key, x, y).block_until_ready()

8.71 ms ± 90.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Predict

In [17]:
from tkm.model import predict
from tkm.metrics import accuracy

y_hat = predict(x, W, feature_map=polynomial, M=12,R=6)
acc = accuracy(y.squeeze(), y_hat)
acc

DeviceArray(0.8922641, dtype=float32)

In [18]:
%timeit predict(x, W, M=12,R=6, feature_map=polynomial)

5.47 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
predict_polynomial = partial(predict, feature_map=polynomial, M=12, R=6)
predict_compiled = jit(predict_polynomial)

%timeit predict_compiled(x,W).block_until_ready()

54.7 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
# from jax import jit
from tkm.model import predict_vmap

y_hat = predict_vmap(x, W, feature_map=polynomial, M=12, R=6)
acc = accuracy(y.squeeze(), y_hat)
print(acc)

%timeit predict_vmap(x, W, feature_map=polynomial, M=12, R=6)

0.8922641
2.26 ms ± 68.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
predict_vmap_poly = partial(predict_vmap, feature_map=polynomial, M=12, R=6)
predict_vmap_compiled = jit(predict_vmap_poly)

%timeit predict_vmap_compiled(x,W).block_until_ready()

53.4 µs ± 554 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### fourier features

In [22]:
from tkm.features import fourier

W = fit(key,x,y,M=12,R=6,lengthscale=0.5,feature_map=fourier)
y_hat = predict(x, W, feature_map=fourier, M=12,R=6,lengthscale=0.5)
acc = accuracy(y.squeeze(), y_hat)
print(acc)

0.89528304


In [23]:
jax.devices()

[GpuDevice(id=0, process_index=0)]

GpuDevice(id=0, process_index=0)